In [17]:
import pandas as pd
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split

# Cargar el dataset
data_path = r"C:\Users\Francesc\Documents\GitHub\won_analysis\datasets\days_to_start_new.csv"
df = pd.read_csv(data_path)

# Mostrar columnas
print("Columnas del DataFrame:")
print(df.columns.tolist())

# Añadir nuevas features
df['day_of_week'] = pd.to_datetime(df['buy_date'], dayfirst=True).dt.dayofweek
df['week_of_year'] = pd.to_datetime(df['buy_date'], dayfirst=True).dt.isocalendar().week

# Función para generar rangos dinámicos de 30 días
def generar_rangos(max_dias):
    intervalo = 30
    bins = list(range(0, max_dias + 1, intervalo)) + [float('inf')]
    labels = [f"{i}-{i+intervalo-1}" for i in range(0, max_dias - intervalo + 1, intervalo)] + [f"{max_dias}+"]
    return bins, labels

# Definir rangos hasta 150 días
max_dias = 150
bins, labels = generar_rangos(max_dias)
df['rango_dias'] = pd.cut(df['days_to_start'], bins=bins, labels=labels, right=False)

# Inspeccionar distribución general
print("\nDistribución de 'rango_dias' en los datos:")
print(df['rango_dias'].value_counts().sort_index())

# Inspeccionar distribución para FS en agosto
fs_agosto = df[(df['Program'] == 'FS') & (df['month_sale'] == 8)]
print("\nDistribución de 'rango_dias' para FS en agosto:")
print(fs_agosto['rango_dias'].value_counts().sort_index())

# Preparar features y target
available_columns = [col for col in ['Program', 'month_sale', 'month_diff', 'null_month', 'one_month', 'two_month', 'three_month', 'four_month', 'five_month', 'six_month', 'seven_month', 'eigth_month'] if col in df.columns]
if 'year_sale' not in df.columns and 'buy_date' in df.columns:
    df['year_sale'] = pd.to_datetime(df['buy_date'], dayfirst=True).dt.year
    available_columns.append('year_sale')
available_columns.extend(['day_of_week', 'week_of_year'])

X = df[available_columns]
X = pd.get_dummies(X, columns=['Program'])
y = df['rango_dias']

# Dividir en entrenamiento y prueba
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Entrenar el modelo Random Forest
model = RandomForestClassifier(n_estimators=200, class_weight='balanced', random_state=42)
model.fit(X_train, y_train)

# Evaluar el modelo
accuracy = model.score(X_test, y_test)
print(f"\nPrecisión del modelo en datos de prueba: {accuracy:.2f}")

# Función para predecir distribución con corrección
def predecir_distribucion_rf(programa, cantidad_ventas, mes, año=2025, max_dias=150):
    bins, labels = generar_rangos(max_dias)
    df['rango_dias'] = pd.cut(df['days_to_start'], bins=bins, labels=labels, right=False)
    
    available_columns = [col for col in ['Program', 'month_sale', 'month_diff', 'null_month', 'one_month', 'two_month', 'three_month', 'four_month', 'five_month', 'six_month', 'seven_month', 'eigth_month'] if col in df.columns]
    if 'year_sale' not in df.columns and 'buy_date' in df.columns:
        df['year_sale'] = pd.to_datetime(df['buy_date'], dayfirst=True).dt.year
        available_columns.append('year_sale')
    available_columns.extend(['day_of_week', 'week_of_year'])
    
    X = df[available_columns]
    X = pd.get_dummies(X, columns=['Program'])
    y = df['rango_dias']
    model.fit(X, y)
    
    nueva_data = pd.DataFrame({
        'Program': [programa],
        'month_sale': [mes],
        'year_sale': [año],
        'month_diff': [0],
        'null_month': [0], 'one_month': [0], 'two_month': [0], 'three_month': [0], 
        'four_month': [0], 'five_month': [0], 'six_month': [0], 'seven_month': [0], 
        'eigth_month': [0],
        'day_of_week': [3],  # Valor medio (miércoles), ajustar si hay datos específicos
        'week_of_year': [mes * 4]  # Aproximación simple
    })
    nueva_data = pd.get_dummies(nueva_data, columns=['Program'])
    nueva_data = nueva_data.reindex(columns=X.columns, fill_value=0)
    
    probs = model.predict_proba(nueva_data)[0]
    prediccion = (probs * cantidad_ventas).round().astype(int)
    
    # Corrección basada en datos históricos del mes
    historico_mes = df[(df['Program'] == programa) & (df['month_sale'] == mes)]['rango_dias'].value_counts(normalize=True).reindex(labels, fill_value=0)
    prediccion_corregida = (prediccion * 0.7 + historico_mes * cantidad_ventas * 0.3).round().astype(int)
    
    # Asegurar que sume exactamente la cantidad
    diferencia = cantidad_ventas - prediccion_corregida.sum()
    if diferencia != 0:
        ajuste = (historico_mes * diferencia).round().astype(int)
        prediccion_corregida += ajuste
        while prediccion_corregida.sum() != cantidad_ventas:
            if prediccion_corregida.sum() > cantidad_ventas:
                idx_max = prediccion_corregida.argmax()
                prediccion_corregida[idx_max] -= 1
            else:
                idx_min = prediccion_corregida.argmin()
                prediccion_corregida[idx_min] += 1
    
    return dict(zip(labels, prediccion_corregida))

# Prueba para FS, 50 ventas, agosto 2025
programa = 'FS'
cantidad_ventas = 50
mes = 8
resultado = predecir_distribucion_rf(programa, cantidad_ventas, mes, max_dias=150)
print(f"\nDistribución predicha para {cantidad_ventas} ventas de {programa} en el mes {mes} (año 2025):")
for rango, ventas in resultado.items():
    print(f"{rango} días: {ventas} ventas")

Columnas del DataFrame:
['Program', 'buy_date', 'cohort_start_date', 'days_to_start', 'month_sale', 'month_start', 'month_diff', 'null_month', 'one_month', 'two_month', 'three_month', 'four_month', 'five_month', 'six_month', 'seven_month', 'eigth_month']

Distribución de 'rango_dias' en los datos:
rango_dias
0-29       136
30-59      386
60-89      232
90-119      91
120-149     35
150+        64
Name: count, dtype: int64

Distribución de 'rango_dias' para FS en agosto:
rango_dias
0-29        4
30-59      32
60-89      14
90-119      3
120-149     0
150+        2
Name: count, dtype: int64

Precisión del modelo en datos de prueba: 0.69

Distribución predicha para 50 ventas de FS en el mes 8 (año 2025):
0-29 días: 19 ventas
30-59 días: 8 ventas
60-89 días: 6 ventas
90-119 días: 11 ventas
120-149 días: 4 ventas
150+ días: 2 ventas


In [18]:
import pandas as pd
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split

# Cargar el dataset
data_path = r"C:\Users\Francesc\Documents\GitHub\won_analysis\datasets\days_to_start_new.csv"
df = pd.read_csv(data_path)

# Mostrar columnas
print("Columnas del DataFrame:")
print(df.columns.tolist())

# Añadir nuevas features
df['day_of_week'] = pd.to_datetime(df['buy_date'], dayfirst=True).dt.dayofweek
df['week_of_year'] = pd.to_datetime(df['buy_date'], dayfirst=True).dt.isocalendar().week

# Función para generar rangos dinámicos de 30 días
def generar_rangos(max_dias):
    intervalo = 30
    bins = list(range(0, max_dias + 1, intervalo)) + [float('inf')]
    labels = [f"{i}-{i+intervalo-1}" for i in range(0, max_dias - intervalo + 1, intervalo)] + [f"{max_dias}+"]
    return bins, labels

# Definir rangos hasta 150 días
max_dias = 150
bins, labels = generar_rangos(max_dias)
df['rango_dias'] = pd.cut(df['days_to_start'], bins=bins, labels=labels, right=False)

# Inspeccionar distribución general
print("\nDistribución de 'rango_dias' en los datos:")
print(df['rango_dias'].value_counts().sort_index())

# Inspeccionar distribución para FS en agosto
fs_agosto = df[(df['Program'] == 'FS') & (df['month_sale'] == 8)]
print("\nDistribución de 'rango_dias' para FS en agosto:")
print(fs_agosto['rango_dias'].value_counts().sort_index())

# Preparar features y target
available_columns = [col for col in ['Program', 'month_sale', 'month_diff', 'null_month', 'one_month', 'two_month', 'three_month', 'four_month', 'five_month', 'six_month', 'seven_month', 'eigth_month'] if col in df.columns]
if 'year_sale' not in df.columns and 'buy_date' in df.columns:
    df['year_sale'] = pd.to_datetime(df['buy_date'], dayfirst=True).dt.year
    available_columns.append('year_sale')
available_columns.extend(['day_of_week', 'week_of_year'])

X = df[available_columns]
X = pd.get_dummies(X, columns=['Program'])
y = df['rango_dias']

# Dividir en entrenamiento y prueba
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Entrenar el modelo Random Forest
model = RandomForestClassifier(n_estimators=200, class_weight='balanced', random_state=42)
model.fit(X_train, y_train)

# Evaluar el modelo
accuracy = model.score(X_test, y_test)
print(f"\nPrecisión del modelo en datos de prueba: {accuracy:.2f}")

# Función para predecir distribución con corrección
def predecir_distribucion_rf(programa, cantidad_ventas, mes, año=2025, max_dias=150):
    bins, labels = generar_rangos(max_dias)
    df['rango_dias'] = pd.cut(df['days_to_start'], bins=bins, labels=labels, right=False)
    
    available_columns = [col for col in ['Program', 'month_sale', 'month_diff', 'null_month', 'one_month', 'two_month', 'three_month', 'four_month', 'five_month', 'six_month', 'seven_month', 'eigth_month'] if col in df.columns]
    if 'year_sale' not in df.columns and 'buy_date' in df.columns:
        df['year_sale'] = pd.to_datetime(df['buy_date'], dayfirst=True).dt.year
        available_columns.append('year_sale')
    available_columns.extend(['day_of_week', 'week_of_year'])
    
    X = df[available_columns]
    X = pd.get_dummies(X, columns=['Program'])
    y = df['rango_dias']
    model.fit(X, y)
    
    nueva_data = pd.DataFrame({
        'Program': [programa],
        'month_sale': [mes],
        'year_sale': [año],
        'month_diff': [0],
        'null_month': [0], 'one_month': [0], 'two_month': [0], 'three_month': [0], 
        'four_month': [0], 'five_month': [0], 'six_month': [0], 'seven_month': [0], 
        'eigth_month': [0],
        'day_of_week': [3],  # Valor medio (miércoles)
        'week_of_year': [mes * 4]  # Aproximación simple
    })
    nueva_data = pd.get_dummies(nueva_data, columns=['Program'])
    nueva_data = nueva_data.reindex(columns=X.columns, fill_value=0)
    
    probs = model.predict_proba(nueva_data)[0]
    prediccion = (probs * cantidad_ventas).round().astype(int)
    
    # Corrección basada en datos históricos del mes
    historico_mes = df[(df['Program'] == programa) & (df['month_sale'] == mes)]['rango_dias'].value_counts(normalize=True).reindex(labels, fill_value=0)
    prediccion_corregida = (prediccion * 0.7 + historico_mes * cantidad_ventas * 0.3).round().astype(int)
    
    # Asegurar que sume exactamente la cantidad
    diferencia = cantidad_ventas - prediccion_corregida.sum()
    if diferencia != 0:
        ajuste = (historico_mes * diferencia).round().astype(int)
        prediccion_corregida += ajuste
        while prediccion_corregida.sum() != cantidad_ventas:
            if prediccion_corregida.sum() > cantidad_ventas:
                idx_max = prediccion_corregida.argmax()
                prediccion_corregida[idx_max] -= 1
            else:
                idx_min = prediccion_corregida.argmin()
                prediccion_corregida[idx_min] += 1
    
    return dict(zip(labels, prediccion_corregida))

# Prueba para FS, 50 ventas, agosto 2025
programa = 'FS'
cantidad_ventas = 50
mes = 8  # Cambiado a agosto
resultado = predecir_distribucion_rf(programa, cantidad_ventas, mes, max_dias=150)
print(f"\nDistribución predicha para {cantidad_ventas} ventas de {programa} en el mes {mes} (año 2025):")
for rango, ventas in resultado.items():
    print(f"{rango} días: {ventas} ventas")

Columnas del DataFrame:
['Program', 'buy_date', 'cohort_start_date', 'days_to_start', 'month_sale', 'month_start', 'month_diff', 'null_month', 'one_month', 'two_month', 'three_month', 'four_month', 'five_month', 'six_month', 'seven_month', 'eigth_month']

Distribución de 'rango_dias' en los datos:
rango_dias
0-29       136
30-59      386
60-89      232
90-119      91
120-149     35
150+        64
Name: count, dtype: int64

Distribución de 'rango_dias' para FS en agosto:
rango_dias
0-29        4
30-59      32
60-89      14
90-119      3
120-149     0
150+        2
Name: count, dtype: int64

Precisión del modelo en datos de prueba: 0.69

Distribución predicha para 50 ventas de FS en el mes 8 (año 2025):
0-29 días: 19 ventas
30-59 días: 8 ventas
60-89 días: 6 ventas
90-119 días: 11 ventas
120-149 días: 4 ventas
150+ días: 2 ventas


In [20]:
import pandas as pd
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split

# Cargar el dataset
data_path = r"C:\Users\Francesc\Documents\GitHub\won_analysis\datasets\days_to_start_new.csv"
df = pd.read_csv(data_path)

# Mostrar columnas
print("Columnas del DataFrame:")
print(df.columns.tolist())

# Añadir nuevas features
df['day_of_week'] = pd.to_datetime(df['buy_date'], dayfirst=True).dt.dayofweek
df['week_of_year'] = pd.to_datetime(df['buy_date'], dayfirst=True).dt.isocalendar().week

# Función para generar rangos dinámicos de 30 días
def generar_rangos(max_dias):
    intervalo = 30
    bins = list(range(0, max_dias + 1, intervalo)) + [float('inf')]
    labels = [f"{i}-{i+intervalo-1}" for i in range(0, max_dias - intervalo + 1, intervalo)] + [f"{max_dias}+"]
    return bins, labels

# Definir rangos hasta 150 días
max_dias = 150
bins, labels = generar_rangos(max_dias)
df['rango_dias'] = pd.cut(df['days_to_start'], bins=bins, labels=labels, right=False)

# Inspeccionar distribución general
print("\nDistribución de 'rango_dias' en los datos:")
print(df['rango_dias'].value_counts().sort_index())

# Inspeccionar distribución para FS en agosto
fs_agosto = df[(df['Program'] == 'FS') & (df['month_sale'] == 8)]
print("\nDistribución de 'rango_dias' para FS en agosto:")
print(fs_agosto['rango_dias'].value_counts().sort_index())

# Preparar features y target
available_columns = [col for col in ['Program', 'month_sale', 'month_diff', 'null_month', 'one_month', 'two_month', 'three_month', 'four_month', 'five_month', 'six_month', 'seven_month', 'eigth_month'] if col in df.columns]
if 'year_sale' not in df.columns and 'buy_date' in df.columns:
    df['year_sale'] = pd.to_datetime(df['buy_date'], dayfirst=True).dt.year
    available_columns.append('year_sale')
available_columns.extend(['day_of_week', 'week_of_year'])

X = df[available_columns]
X = pd.get_dummies(X, columns=['Program'])
y = df['rango_dias']

# Dividir en entrenamiento y prueba
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Entrenar el modelo Random Forest
model = RandomForestClassifier(n_estimators=200, class_weight='balanced', random_state=42)
model.fit(X_train, y_train)

# Evaluar el modelo
accuracy = model.score(X_test, y_test)
print(f"\nPrecisión del modelo en datos de prueba: {accuracy:.2f}")

# Función para predecir distribución con corrección
def predecir_distribucion_rf(programa, cantidad_ventas, mes, año=2025, max_dias=150):
    bins, labels = generar_rangos(max_dias)
    df['rango_dias'] = pd.cut(df['days_to_start'], bins=bins, labels=labels, right=False)
    
    available_columns = [col for col in ['Program', 'month_sale', 'month_diff', 'null_month', 'one_month', 'two_month', 'three_month', 'four_month', 'five_month', 'six_month', 'seven_month', 'eigth_month'] if col in df.columns]
    if 'year_sale' not in df.columns and 'buy_date' in df.columns:
        df['year_sale'] = pd.to_datetime(df['buy_date'], dayfirst=True).dt.year
        available_columns.append('year_sale')
    available_columns.extend(['day_of_week', 'week_of_year'])
    
    X = df[available_columns]
    X = pd.get_dummies(X, columns=['Program'])
    y = df['rango_dias']
    model.fit(X, y)
    
    nueva_data = pd.DataFrame({
        'Program': [programa],
        'month_sale': [mes],
        'year_sale': [año],
        'month_diff': [0],
        'null_month': [0], 'one_month': [0], 'two_month': [0], 'three_month': [0], 
        'four_month': [0], 'five_month': [0], 'six_month': [0], 'seven_month': [0], 
        'eigth_month': [0],
        'day_of_week': [3],  # Valor medio (miércoles)
        'week_of_year': [mes * 4]  # Aproximación simple
    })
    nueva_data = pd.get_dummies(nueva_data, columns=['Program'])
    nueva_data = nueva_data.reindex(columns=X.columns, fill_value=0)
    
    probs = model.predict_proba(nueva_data)[0]
    prediccion = (probs * cantidad_ventas).round().astype(int)
    
    # Corrección con mayor peso al histórico (50% modelo + 50% histórico)
    historico_mes = df[(df['Program'] == programa) & (df['month_sale'] == mes)]['rango_dias'].value_counts(normalize=True).reindex(labels, fill_value=0)
    prediccion_corregida = (prediccion * 0.5 + historico_mes * cantidad_ventas * 0.5).round().astype(int)
    
    # Asegurar que sume exactamente la cantidad
    diferencia = cantidad_ventas - prediccion_corregida.sum()
    if diferencia != 0:
        ajuste = (historico_mes * diferencia).round().astype(int)
        prediccion_corregida += ajuste
        while prediccion_corregida.sum() != cantidad_ventas:
            if prediccion_corregida.sum() > cantidad_ventas:
                idx_max = prediccion_corregida.argmax()
                prediccion_corregida[idx_max] -= 1
            else:
                idx_min = prediccion_corregida.argmin()
                prediccion_corregida[idx_min] += 1
    
    return dict(zip(labels, prediccion_corregida))

# Prueba para FS, 50 ventas, agosto 2025
programa = 'FS'
cantidad_ventas = 50
mes = 8
resultado = predecir_distribucion_rf(programa, cantidad_ventas, mes, max_dias=150)
print(f"\nDistribución predicha para {cantidad_ventas} ventas de {programa} en el mes {mes} (año 2025):")
for rango, ventas in resultado.items():
    print(f"{rango} días: {ventas} ventas")

Columnas del DataFrame:
['Program', 'buy_date', 'cohort_start_date', 'days_to_start', 'month_sale', 'month_start', 'month_diff', 'null_month', 'one_month', 'two_month', 'three_month', 'four_month', 'five_month', 'six_month', 'seven_month', 'eigth_month']

Distribución de 'rango_dias' en los datos:
rango_dias
0-29       136
30-59      386
60-89      232
90-119      91
120-149     35
150+        64
Name: count, dtype: int64

Distribución de 'rango_dias' para FS en agosto:
rango_dias
0-29        4
30-59      32
60-89      14
90-119      3
120-149     0
150+        2
Name: count, dtype: int64

Precisión del modelo en datos de prueba: 0.69

Distribución predicha para 50 ventas de FS en el mes 8 (año 2025):
0-29 días: 14 ventas
30-59 días: 16 ventas
60-89 días: 8 ventas
90-119 días: 8 ventas
120-149 días: 2 ventas
150+ días: 2 ventas


In [23]:
import pandas as pd
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split

# Cargar el dataset
data_path = r"C:\Users\Francesc\Documents\GitHub\won_analysis\datasets\days_to_start_new.csv"
df = pd.read_csv(data_path)

# Mostrar columnas
print("Columnas del DataFrame:")
print(df.columns.tolist())

# Añadir nuevas features
df['day_of_week'] = pd.to_datetime(df['buy_date'], dayfirst=True).dt.dayofweek
df['week_of_year'] = pd.to_datetime(df['buy_date'], dayfirst=True).dt.isocalendar().week

# Función para generar rangos dinámicos de 30 días
def generar_rangos(max_dias):
    intervalo = 30
    bins = list(range(0, max_dias + 1, intervalo)) + [float('inf')]
    labels = [f"{i}-{i+intervalo-1}" for i in range(0, max_dias - intervalo + 1, intervalo)] + [f"{max_dias}+"]
    return bins, labels

# Definir rangos hasta 150 días
max_dias = 150
bins, labels = generar_rangos(max_dias)
df['rango_dias'] = pd.cut(df['days_to_start'], bins=bins, labels=labels, right=False)

# Inspeccionar distribución general
print("\nDistribución de 'rango_dias' en los datos:")
print(df['rango_dias'].value_counts().sort_index())

# Inspeccionar distribución para FS en agosto
fs_agosto = df[(df['Program'] == 'FS') & (df['month_sale'] == 8)]
print("\nDistribución de 'rango_dias' para FS en agosto:")
print(fs_agosto['rango_dias'].value_counts().sort_index())

# Preparar features y target
available_columns = [col for col in ['Program', 'month_sale', 'month_diff', 'null_month', 'one_month', 'two_month', 'three_month', 'four_month', 'five_month', 'six_month', 'seven_month', 'eigth_month'] if col in df.columns]
if 'year_sale' not in df.columns and 'buy_date' in df.columns:
    df['year_sale'] = pd.to_datetime(df['buy_date'], dayfirst=True).dt.year
    available_columns.append('year_sale')
available_columns.extend(['day_of_week', 'week_of_year'])

X = df[available_columns]
X = pd.get_dummies(X, columns=['Program'])
y = df['rango_dias']

# Dividir en entrenamiento y prueba
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Entrenar el modelo Random Forest
model = RandomForestClassifier(n_estimators=200, class_weight='balanced', random_state=42)
model.fit(X_train, y_train)

# Evaluar el modelo
accuracy = model.score(X_test, y_test)
print(f"\nPrecisión del modelo en datos de prueba: {accuracy:.2f}")

# Función para predecir distribución con corrección
def predecir_distribucion_rf(programa, cantidad_ventas, mes, año=2025, max_dias=150):
    bins, labels = generar_rangos(max_dias)
    df['rango_dias'] = pd.cut(df['days_to_start'], bins=bins, labels=labels, right=False)
    
    available_columns = [col for col in ['Program', 'month_sale', 'month_diff', 'null_month', 'one_month', 'two_month', 'three_month', 'four_month', 'five_month', 'six_month', 'seven_month', 'eigth_month'] if col in df.columns]
    if 'year_sale' not in df.columns and 'buy_date' in df.columns:
        df['year_sale'] = pd.to_datetime(df['buy_date'], dayfirst=True).dt.year
        available_columns.append('year_sale')
    available_columns.extend(['day_of_week', 'week_of_year'])
    
    X = df[available_columns]
    X = pd.get_dummies(X, columns=['Program'])
    y = df['rango_dias']
    model.fit(X, y)
    
    nueva_data = pd.DataFrame({
        'Program': [programa],
        'month_sale': [mes],
        'year_sale': [año],
        'month_diff': [0],
        'null_month': [0], 'one_month': [0], 'two_month': [0], 'three_month': [0], 
        'four_month': [0], 'five_month': [0], 'six_month': [0], 'seven_month': [0], 
        'eigth_month': [0],
        'day_of_week': [3],  # Valor medio (miércoles)
        'week_of_year': [mes * 4]  # Aproximación simple
    })
    nueva_data = pd.get_dummies(nueva_data, columns=['Program'])
    nueva_data = nueva_data.reindex(columns=X.columns, fill_value=0)
    
    probs = model.predict_proba(nueva_data)[0]
    prediccion = (probs * cantidad_ventas).round().astype(int)
    
    # Corrección con mayor peso al histórico (20% modelo + 80% histórico)
    historico_mes = df[(df['Program'] == programa) & (df['month_sale'] == mes)]['rango_dias'].value_counts(normalize=True).reindex(labels, fill_value=0)
    prediccion_corregida = (prediccion * 0.2 + historico_mes * cantidad_ventas * 0.8).round().astype(int)
    
    # Asegurar que sume exactamente la cantidad
    diferencia = cantidad_ventas - prediccion_corregida.sum()
    if diferencia != 0:
        ajuste = (historico_mes * diferencia).round().astype(int)
        prediccion_corregida += ajuste
        while prediccion_corregida.sum() != cantidad_ventas:
            if prediccion_corregida.sum() > cantidad_ventas:
                idx_max = prediccion_corregida.argmax()
                prediccion_corregida[idx_max] -= 1
            else:
                idx_min = prediccion_corregida.argmin()
                prediccion_corregida[idx_min] += 1
    
    return dict(zip(labels, prediccion_corregida))

# Prueba para FS, 50 ventas, agosto 2025
programa = 'FS'
cantidad_ventas = 50
mes = 8
resultado = predecir_distribucion_rf(programa, cantidad_ventas, mes, max_dias=150)
print(f"\nDistribución predicha para {cantidad_ventas} ventas de {programa} en el mes {mes} (año 2025):")
for rango, ventas in resultado.items():
    print(f"{rango} días: {ventas} ventas")

Columnas del DataFrame:
['Program', 'buy_date', 'cohort_start_date', 'days_to_start', 'month_sale', 'month_start', 'month_diff', 'null_month', 'one_month', 'two_month', 'three_month', 'four_month', 'five_month', 'six_month', 'seven_month', 'eigth_month']

Distribución de 'rango_dias' en los datos:
rango_dias
0-29       136
30-59      386
60-89      232
90-119      91
120-149     35
150+        64
Name: count, dtype: int64

Distribución de 'rango_dias' para FS en agosto:
rango_dias
0-29        4
30-59      32
60-89      14
90-119      3
120-149     0
150+        2
Name: count, dtype: int64

Precisión del modelo en datos de prueba: 0.69

Distribución predicha para 50 ventas de FS en el mes 8 (año 2025):
0-29 días: 8 ventas
30-59 días: 23 ventas
60-89 días: 11 ventas
90-119 días: 5 ventas
120-149 días: 1 ventas
150+ días: 2 ventas


In [7]:
import pandas as pd
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split

# Cargar el dataset
data_path = r"C:\Users\Francesc\Documents\GitHub\won_analysis\datasets\days_to_start_new.csv"
df = pd.read_csv(data_path)

# Mostrar columnas
print("Columnas del DataFrame:")
print(df.columns.tolist())

# Añadir nuevas features
df['day_of_week'] = pd.to_datetime(df['buy_date'], dayfirst=True).dt.dayofweek
df['week_of_year'] = pd.to_datetime(df['buy_date'], dayfirst=True).dt.isocalendar().week

# Función para generar rangos dinámicos de 30 días
def generar_rangos(max_dias):
    intervalo = 30
    bins = list(range(0, max_dias + 1, intervalo)) + [float('inf')]
    labels = [f"{i}-{i+intervalo-1}" for i in range(0, max_dias - intervalo + 1, intervalo)] + [f"{max_dias}+"]
    return bins, labels

# Definir rangos hasta 150 días
max_dias = 150
bins, labels = generar_rangos(max_dias)
df['rango_dias'] = pd.cut(df['days_to_start'], bins=bins, labels=labels, right=False)

# Preparar features y target
available_columns = [col for col in ['Program', 'month_sale', 'month_diff', 'null_month', 'one_month', 'two_month', 'three_month', 'four_month', 'five_month', 'six_month', 'seven_month', 'eigth_month'] if col in df.columns]
if 'year_sale' not in df.columns and 'buy_date' in df.columns:
    df['year_sale'] = pd.to_datetime(df['buy_date'], dayfirst=True).dt.year
    available_columns.append('year_sale')
available_columns.extend(['day_of_week', 'week_of_year'])

X = df[available_columns]
X = pd.get_dummies(X, columns=['Program'])
y = df['rango_dias']

# Dividir en entrenamiento y prueba
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Entrenar el modelo Random Forest
model = RandomForestClassifier(n_estimators=200, class_weight='balanced', random_state=42)
model.fit(X_train, y_train)

# Evaluar el modelo
accuracy = model.score(X_test, y_test)
print(f"\nPrecisión del modelo en datos de prueba: {accuracy:.2f}")

# Función para predecir distribución con enfoque híbrido 80-20
def predecir_distribucion_rf(programa, cantidad_ventas, mes, año=2025, max_dias=150):
    bins, labels = generar_rangos(max_dias)
    df['rango_dias'] = pd.cut(df['days_to_start'], bins=bins, labels=labels, right=False)
    
    # Inspeccionar distribución histórica para el programa y mes
    historico = df[(df['Program'] == programa) & (df['month_sale'] == mes)]
    print(f"\nDistribución histórica de 'rango_dias' para {programa} en el mes {mes}:")
    print(historico['rango_dias'].value_counts().sort_index())
    
    # Preparar datos para el modelo
    available_columns = [col for col in ['Program', 'month_sale', 'month_diff', 'null_month', 'one_month', 'two_month', 'three_month', 'four_month', 'five_month', 'six_month', 'seven_month', 'eigth_month'] if col in df.columns]
    if 'year_sale' not in df.columns and 'buy_date' in df.columns:
        df['year_sale'] = pd.to_datetime(df['buy_date'], dayfirst=True).dt.year
        available_columns.append('year_sale')
    available_columns.extend(['day_of_week', 'week_of_year'])
    
    X = df[available_columns]
    X = pd.get_dummies(X, columns=['Program'])
    y = df['rango_dias']
    model.fit(X, y)
    
    nueva_data = pd.DataFrame({
        'Program': [programa],
        'month_sale': [mes],
        'year_sale': [año],
        'month_diff': [0],
        'null_month': [0], 'one_month': [0], 'two_month': [0], 'three_month': [0], 
        'four_month': [0], 'five_month': [0], 'six_month': [0], 'seven_month': [0], 
        'eigth_month': [0],
        'day_of_week': [3],  # Valor medio (miércoles)
        'week_of_year': [mes * 4]  # Aproximación simple
    })
    nueva_data = pd.get_dummies(nueva_data, columns=['Program'])
    nueva_data = nueva_data.reindex(columns=X.columns, fill_value=0)
    
    probs = model.predict_proba(nueva_data)[0]
    prediccion = (probs * cantidad_ventas).round().astype(int)
    
    # Corrección con 80% histórico + 20% modelo
    historico_mes = historico['rango_dias'].value_counts(normalize=True).reindex(labels, fill_value=0)
    prediccion_corregida = (prediccion * 0.2 + historico_mes * cantidad_ventas * 0.8).round().astype(int)
    
    # Asegurar que sume exactamente la cantidad
    diferencia = cantidad_ventas - prediccion_corregida.sum()
    if diferencia != 0:
        ajuste = (historico_mes * diferencia).round().astype(int)
        prediccion_corregida += ajuste
        while prediccion_corregida.sum() != cantidad_ventas:
            if prediccion_corregida.sum() > cantidad_ventas:
                idx_max = prediccion_corregida.argmax()
                prediccion_corregida[idx_max] -= 1
            else:
                idx_min = prediccion_corregida.argmin()
                prediccion_corregida[idx_min] += 1
    
    return dict(zip(labels, prediccion_corregida))

# Prueba para FS, 50 ventas, agosto 2025
programa = 'FS'
cantidad_ventas = 50
mes = 5
resultado = predecir_distribucion_rf(programa, cantidad_ventas, mes, max_dias=150)
print(f"\nDistribución predicha para {cantidad_ventas} ventas de {programa} en el mes {mes} (año 2025):")
for rango, ventas in resultado.items():
    print(f"{rango} días: {ventas} ventas")

Columnas del DataFrame:
['Program', 'buy_date', 'cohort_start_date', 'days_to_start', 'month_sale', 'month_start', 'month_diff', 'null_month', 'one_month', 'two_month', 'three_month', 'four_month', 'five_month', 'six_month', 'seven_month', 'eigth_month']

Precisión del modelo en datos de prueba: 0.69

Distribución histórica de 'rango_dias' para FS en el mes 5:
rango_dias
0-29       16
30-59      30
60-89       3
90-119     10
120-149     7
150+        8
Name: count, dtype: int64

Distribución predicha para 50 ventas de FS en el mes 5 (año 2025):
0-29 días: 14 ventas
30-59 días: 16 ventas
60-89 días: 2 ventas
90-119 días: 8 ventas
120-149 días: 5 ventas
150+ días: 5 ventas


C:\Users\Francesc\AppData\Local\Temp\ipykernel_28724\180191755.py:102: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  prediccion_corregida[idx_max] -= 1
C:\Users\Francesc\AppData\Local\Temp\ipykernel_28724\180191755.py:102: FutureWarning: Series.__setitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To set a value by position, use `ser.iloc[pos] = value`
  prediccion_corregida[idx_max] -= 1
